In [1]:
%matplotlib inline
%config InlineBackend.figure_format='svg'


import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from morphing_birds import Hawk3D


hawk3d = Hawk3D("../data/mean_hawk_shape.csv")

In [2]:
# %matplotlib notebook
keypoints = np.zeros([1,4,3])
hawk3d.display_hawk()


IntSlider(value=60, description='azimuth', max=90, min=-90, step=5)

IntSlider(value=20, description='elevation', max=90, min=-90, step=5)

Output()

In [3]:
hawk3d.display_hawk()

AttributeError: 'HawkPlotter' object has no attribute 'display_hawk'